### Simple Cifar10 with PySyft clients using WebSocketWorker Class in PySyft

We first deploy the workers/clients using the other python script in the directory

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging
import sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter("%(asctime)s %(levelname)s %(filename)s(l:%(lineno)d) - %(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]

In [2]:
import syft as sy
from syft import WebsocketClientWorker
from syft.frameworks.torch.fl import utils

hook = sy.TorchHook(torch)

/home/anirban/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anirban/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anirban/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anirban/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

Here we connect to our three worker using their ids and ports. 

**Note:** Wait few seconds before running this cell as the deployment may take some time.

In [3]:
client1 = WebsocketClientWorker(hook=hook, id="client0", host='localhost', port=5000, verbose=True)
client2 = WebsocketClientWorker(hook=hook, id="client1", host='localhost', port=5001, verbose=True)
client3 = WebsocketClientWorker(hook=hook, id="client2", host='localhost', port=5002, verbose=True)

### Now we follow the tutorial at [PySyft Cifar10 Tutorial](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/advanced/Federated%20CIFAR10.ipynb)

In [4]:
epochs = 10
class Arguments():
    def __init__(self):
        global epochs
        self.batch_size = 256
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 42
        self.log_interval = 200
        self.save_model = False
        self.federate_after_n_batches = 20

args = Arguments()
use_cuda = False #not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Define the model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

## Train and test functions

In [6]:
def train_on_batches(worker, batches, model_in, device, lr, log_interval=25):
    """Train the model on the worker on the provided batches
    Args:
        worker(syft.workers.BaseWorker): worker on which the
        training will be executed
        batches: batches of data of this worker
        model_in: machine learning model, training will be done on a copy
        device (torch.device): where to run the training
        lr: learning rate of the training steps
    Returns:
        model, loss: obtained model and loss after training
    """
    model = model_in.copy()
    optimizer = optim.SGD(model.parameters(), lr=lr)  # TODO momentum is not supported at the moment

    model.train()
    model.send(worker)
    loss_local = False

    for batch_idx, (data, target) in enumerate(batches):
        loss_local = False
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            loss = loss.get()  # <-- NEW: get the loss back
            loss_local = True
            logger.debug(
                "Train Worker {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    worker.id,
                    batch_idx,
                    len(batches),
                    100.0 * batch_idx / len(batches),
                    loss.item(),
                )
            )

    if not loss_local:
        loss = loss.get()  # <-- NEW: get the loss back
    model.get()  # <-- NEW: get the model back
    return model, loss


def get_next_batches(fdataloader: sy.FederatedDataLoader, nr_batches: int):
    """retrieve next nr_batches of the federated data loader and group
    the batches by worker
    Args:
        fdataloader (sy.FederatedDataLoader): federated data loader
        over which the function will iterate
        nr_batches (int): number of batches (per worker) to retrieve
    Returns:
        Dict[syft.workers.BaseWorker, List[batches]]
    """
    batches = {}
    for worker_id in fdataloader.workers:
        worker = fdataloader.federated_dataset.datasets[worker_id].location
        batches[worker] = []
    try:
        for i in range(nr_batches):
            next_batches = next(fdataloader)
            for worker in next_batches:
                batches[worker].append(next_batches[worker])
    except StopIteration:
        pass
    return batches


def train(model, device, federated_train_loader, lr, federate_after_n_batches, abort_after_one=False):
    model.train()

    nr_batches = federate_after_n_batches

    models = {}
    loss_values = {}

    iter(federated_train_loader)  # initialize iterators
    batches = get_next_batches(federated_train_loader, nr_batches)
    counter = 0

    while True:
        logger.debug(f"Starting training round, batches [{counter}, {counter + nr_batches}]")
        data_for_all_workers = True
        for worker in batches:
            curr_batches = batches[worker]
            if curr_batches:
                models[worker], loss_values[worker] = train_on_batches(
                    worker, curr_batches, model, device, lr
                )
            else:
                data_for_all_workers = False
        counter += nr_batches
        if not data_for_all_workers:
            logger.debug("At least one worker ran out of data, stopping.")
            break

        model = utils.federated_avg(models)
        batches = get_next_batches(federated_train_loader, nr_batches)
        if abort_after_one:
            break
    return model


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    logger.debug("\n")
    accuracy = 100.0 * correct / len(test_loader.dataset)
    logger.info(
        "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), accuracy
        )
    )

## Load the federated train loader and the test loader

In [7]:
'''<--Load CIFAR dataset from torch vision module distribute to workers using PySyft's Federated Data loader'''
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
                datasets.CIFAR10('../data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ]))
                                .federate((client1, client2, client3)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
                                batch_size=args.batch_size, shuffle=True, iter_per_worker=True)

test_loader = torch.utils.data.DataLoader(
                            datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                           ])),
                            batch_size=args.test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [8]:
model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Starting the training

In [9]:
for epoch in range(1, args.epochs + 1):
    print("Starting epoch {}/{}".format(epoch, args.epochs))
    model = train(model, device, federated_train_loader, args.lr, args.federate_after_n_batches)
    test(model, device, test_loader)

Starting epoch 1/10


2020-07-04 02:13:13,194 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [0, 20]
DEBUG:__main__:Starting training round, batches [0, 20]
2020-07-04 02:13:13,496 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 2.302377
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 2.302377
2020-07-04 02:13:17,124 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 2.296749
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 2.296749
2020-07-04 02:13:21,003 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 2.310817
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 2.310817
2020-07-04 02:13:37,304 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [20, 40]
DEBUG:__main__:Starting training round, batches [20, 40]
2020-07-04 02:13:37,546 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 2.304486
DE

2020-07-04 02:17:41,421 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 2.302332
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 2.302332
2020-07-04 02:17:44,916 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 2.289783
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 2.289783
2020-07-04 02:18:00,388 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [240, 260]
DEBUG:__main__:Starting training round, batches [240, 260]
2020-07-04 02:18:00,642 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 2.306515
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 2.306515
2020-07-04 02:18:04,022 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 2.300251
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 2.300251
2020-07-04 02:18:07,846 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%

Starting epoch 2/10


2020-07-04 02:18:28,908 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [0, 20]
DEBUG:__main__:Starting training round, batches [0, 20]
2020-07-04 02:18:29,141 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 2.310410
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 2.310410
2020-07-04 02:18:32,947 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 2.299890
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 2.299890
2020-07-04 02:18:36,622 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 2.296621
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 2.296621
2020-07-04 02:18:52,249 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [20, 40]
DEBUG:__main__:Starting training round, batches [20, 40]
2020-07-04 02:18:52,447 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 2.298420
DE

...

2020-07-04 03:00:36,218 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [0, 20]
DEBUG:__main__:Starting training round, batches [0, 20]
2020-07-04 03:00:36,452 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.560614
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 1.560614
2020-07-04 03:00:42,523 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 1.761739
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 1.761739
2020-07-04 03:00:47,523 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 1.851194
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 1.851194
2020-07-04 03:01:10,131 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [20, 40]
DEBUG:__main__:Starting training round, batches [20, 40]
2020-07-04 03:01:10,335 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.761449
DE

2020-07-04 03:06:25,728 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 2.027709
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 2.027709
2020-07-04 03:06:29,232 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 1.782233
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 1.782233
2020-07-04 03:06:44,649 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [240, 260]
DEBUG:__main__:Starting training round, batches [240, 260]
2020-07-04 03:06:44,837 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.692402
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 1.692402
2020-07-04 03:06:48,134 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 1.543371
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 1.543371
2020-07-04 03:06:51,726 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%

Starting epoch 10/10


2020-07-04 03:07:12,387 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [0, 20]
DEBUG:__main__:Starting training round, batches [0, 20]
2020-07-04 03:07:12,550 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.828359
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 1.828359
2020-07-04 03:07:15,826 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 1.701957
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 1.701957
2020-07-04 03:07:19,061 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 1.691286
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 1.691286
2020-07-04 03:07:35,210 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [20, 40]
DEBUG:__main__:Starting training round, batches [20, 40]
2020-07-04 03:07:35,446 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.627628
DE

2020-07-04 03:11:27,736 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 1.705253
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 1.705253
2020-07-04 03:11:31,219 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%)]	Loss: 1.902021
DEBUG:__main__:Train Worker client2: [0/20 (0%)]	Loss: 1.902021
2020-07-04 03:11:46,753 DEBUG <ipython-input-6-3889d40c5170>(l:84) - Starting training round, batches [240, 260]
DEBUG:__main__:Starting training round, batches [240, 260]
2020-07-04 03:11:46,954 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client0: [0/20 (0%)]	Loss: 1.597477
DEBUG:__main__:Train Worker client0: [0/20 (0%)]	Loss: 1.597477
2020-07-04 03:11:50,132 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client1: [0/20 (0%)]	Loss: 1.593767
DEBUG:__main__:Train Worker client1: [0/20 (0%)]	Loss: 1.593767
2020-07-04 03:11:53,537 DEBUG <ipython-input-6-3889d40c5170>(l:37) - Train Worker client2: [0/20 (0%